In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
from sklearn import metrics
np.random.seed(0)
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [2]:
def toKaggle(filename,prSoft):
    fh = open(filename,'w') 
    fh.write('ID,Target\n')
    for i,yi in enumerate(prSoft[:,1].ravel()):
        fh.write('{},{}\n'.format(i+1,yi)) # output each prediction
    fh.close() 

In [3]:
# dummy class to simplify AUC computation
class dummy(ml.classifier): 
    def set(self,P):
        self.Pr = P
        self.classes = np.array([0,1]) 
    def predictSoft(self, X):
        return self.Pr

In [4]:
# load data
X = np.genfromtxt('data/X_train.txt', delimiter=None) 
Y = np.genfromtxt('data/Y_train.txt', delimiter=None) 

# print(X.shape)
# X,Y = ml.shuffleData(X,Y)

# # select the first 30,000 (shuffled) data points as training data.
# Xtr = X[0:50000,:,]   
# Ytr = Y[0:50000,]  

# # next 30,000 data points as validation data.
# Xva = X[50000:100000,:,]
# Yva = Y[50000:100000,]

Xtr,Xva,Ytr,Yva = ml.splitData(X,Y,0.8)

learner = RandomForestClassifier(n_estimators=200, max_depth=19, random_state=0)
learner.fit(Xtr, Ytr)

print("score = " + str(learner.score(Xva, Yva)))


score = 0.72715


In [5]:
Xte = np.genfromtxt('data/X_test.txt', delimiter=None)

# Create csv format
Pv1 = np.zeros((Xva.shape[0],2))
Pe1 = np.zeros((Xte.shape[0],2))
ens = dummy() # make dummy learner for computing auc values

pv1 = learner.predict_proba(Xva)
pe1 = learner.predict_proba(Xte)

toKaggle('Pv1.csv',Pv1)
toKaggle('Pe1.csv',Pe1)
ens.set(Pv1);

# Create submission
Xte = np.genfromtxt('data/X_test.txt', delimiter=None)
print(Xte.shape[0])
Yte = np.vstack((np.arange(Xte.shape[0]), learner.predict_proba(Xte)[:,1])).T
# Output a file with two columns, a row ID and a confidence in class 1:
np.savetxt('Y_submit.txt',Yte,'%d, %.2f',header='ID,Prob1',comments='',delimiter=',')


100000
